In [ ]:
! pip install nltk
import nltk
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
stop_words = nltk.corpus.stopwords.words(['english'])
nltk.download('omw-1.4')
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:

nltk.download( 'vader_lexicon' )
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiment=SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
import pandas as pd
import re

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df=pd.read_csv(r'/content/gdrive/MyDrive/capstone project/modified datasets/CleanedDataset.csv')

In [ ]:
df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text
0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605.0,4838.0,False,2021-02-10,Blue Ridge Bank shares halted by NYSE after AT...
1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532.0,25483.0,False,2021-02-10,"😎 Today, that's this Thursday, we will do a ""🎬..."
2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332.0,924.0,False,2021-02-10,"Guys evening, I have read this article about B..."
3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129.0,14.0,False,2021-02-10,$BTC A big chance in a billion! Price: \487264...
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472.0,10482.0,False,2021-02-10,This network is secured by 9 508 nodes as of t...


In [ ]:
SlangDict_df=pd.read_csv(r'/content/gdrive/MyDrive/capstone project/datasets/additional/SlangData.csv')

In [ ]:
SlangDict_df.shape

(1481, 3)

In [ ]:
SLANG_DICT={}
for i in range(1,SlangDict_df.shape[0]):
  SLANG_DICT[SlangDict_df["Slang"][i]]=SlangDict_df["Meaning"][i]

In [ ]:
SLANG_DICT

In [ ]:
def cleanText(string):
  try:
    fin=""
    string=string.replace('\n'," ")
    #print(string)
    string=string.split()    
    for  i in range(len(string)):
      if (string[i]=="") or (string[i][0] in "@/") or ("http" in string[i]) or (string[i][0] =="#" and string[i][-1] not in "?!.,"):
        continue
      else:
        if((string[i][0] =="#" and string[i][-1] in "?!.,")):
          fin+=" "+string[i][1:]
        else:
          fin+=" "+string[i]
    return fin.strip()
  except Exception as e:
    print("ERROR"," ".join(string))
    print(e)

In [ ]:
def findInDict(word):
  try:
    SLANG_DICT[word]
    return 1
  except:
    return 0

In [ ]:
def convertSlang(string):

  string=string.split()
  for i in range(len(string)):
    s = re.sub(r'[^\w\s]','',string[i])
    if(findInDict(s.lower())==1):
      string[i]=string[i].replace(s,SLANG_DICT[s.lower()])

  return " ".join(string)

In [ ]:
cln=[cleanText(i) for i in df.text]

In [ ]:
clean_wo_slang=[sentiment.polarity_scores(i) for i in cln]
clean_with_slang=[sentiment.polarity_scores(convertSlang(i)) for i in cln]

In [ ]:
text_lemm = [lem.lemmatize(t) for t in cln]

sentiment_lemm=[sentiment.polarity_scores(i) for i in text_lemm]
sentiment_lemm_slang=[sentiment.polarity_scores(convertSlang(i)) for i in text_lemm]
score_df = pd.DataFrame(
    {'not lemmatized without slang': clean_wo_slang,
     'not lemmatized with slang': clean_with_slang,
     'lemmatized without slang': sentiment_lemm,
     'lemmatized with slang': sentiment_lemm_slang,
    })
neutral=sentiment.polarity_scores("hsssvsb")
for i in score_df:
  count=0
  for j in score_df[i]:
    if j==neutral:
      count+=1
  print(i," : ",count)


not lemmatized without slang  :  964194
not lemmatized with slang  :  871447
lemmatized without slang  :  964185
lemmatized with slang  :  871437


In [ ]:
score=[i["compound"] for i in score_df["lemmatized with slang"]]

In [ ]:
#score[:5]

[0.296, 0.4939, 0.5719, 0.3164, 0.4019]

In [ ]:
df["score"]=score
df["text"]=cln



In [ ]:
df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,score
0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605.0,4838.0,False,2021-02-10,Blue Ridge Bank shares halted by NYSE after AT...,0.2960
1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532.0,25483.0,False,2021-02-10,"😎 Today, that's this Thursday, we will do a ""🎬...",0.4939
2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332.0,924.0,False,2021-02-10,"Guys evening, I have read this article about B...",0.5719
3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129.0,14.0,False,2021-02-10,$BTC A big chance in a billion! Price: \487264...,0.3164
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472.0,10482.0,False,2021-02-10,This network is secured by 9 508 nodes as of t...,0.4019


In [ ]:
df.to_csv(r'/content/gdrive/MyDrive/capstone project/modified datasets/CleanedDataset.csv',index=False)

In [ ]:
score_df.to_csv(r'/content/gdrive/MyDrive/capstone project/datasets/additional/ScoreData.csv',index=False)

In [ ]:
"""sentiment_lemm=[sentiment.polarity_scores(i) for i in text_lemm]
#sentiment_lemm=[str(i) for i in sentiment_lemm]"""

'sentiment_lemm=[sentiment.polarity_scores(i) for i in text_lemm]\n#sentiment_lemm=[str(i) for i in sentiment_lemm]'

In [ ]:
"""sentiment_lemm_slang=[sentiment.polarity_scores(convertSlang(i)) for i in text_lemm]
#sentiment_lemm_slang=[str(i) for i in sentiment_lemm_slang]"""

'sentiment_lemm_slang=[sentiment.polarity_scores(convertSlang(i)) for i in text_lemm]\n#sentiment_lemm_slang=[str(i) for i in sentiment_lemm_slang]'

In [ ]:
"""score_df = pd.DataFrame(
    {'not lemmatized without slang': clean_wo_slang,
     'not lemmatized with slang': clean_with_slang,
     'lemmatized without slang': sentiment_lemm,
     'lemmatized with slang': sentiment_lemm_slang,
    })"""

"score_df = pd.DataFrame(\n    {'not lemmatized without slang': clean_wo_slang,\n     'not lemmatized with slang': clean_with_slang,\n     'lemmatized without slang': sentiment_lemm,\n     'lemmatized with slang': sentiment_lemm_slang,\n    })"

In [ ]:
"""neutral=sentiment.polarity_scores("hsssvsb")"""

'neutral=sentiment.polarity_scores("hsssvsb")'

In [ ]:
"""for i in score_df:
  count=0
  for j in score_df[i]:
    if j==neutral:
      count+=1
  print(i," : ",count)"""

'for i in score_df:\n  count=0\n  for j in score_df[i]:\n    if j==neutral:\n      count+=1\n  print(i," : ",count)'

In [ ]:
#df.to_csv(r'/content/gdrive/MyDrive/capstone project/modified datasets/CleanedDataset.csv')